In [1]:
from bokeh.plotting import figure, curdoc
from bokeh.io import output_notebook,output_file, show
from bokeh.models.glyphs import ImageURL
from bokeh.models import HoverTool, Legend, CustomJS
from bokeh.models.sources import ColumnDataSource
import calendar
# from bokeh.palettes import Category20
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
output_notebook()
# Output to file
output_file("first_viz.html")

Loading BokehJS ...

In [2]:
COLORS = ['#7DFB6D', '#C7B815', '#D4752E', '#C7583F','#d64161', '#86af49', '#c1946a',
          '#92a8d1','#563f46', '#96897f', '#7e4a35', '#c1502e','#587e76', '#87bdd8',
          '#D4752E','#c83349','#7DFB6D', '#C7B815', '#622569']

<div class="alert alert-block alert-success">
    <b>Data preprocessing</b>
</div>

In [3]:
data=pd.read_csv('AllBirdsv4.csv')
data.shape

(2081, 8)

In [4]:
data.head()

,File ID,English_name,Vocalization_type,Quality,Time,Date,X,Y
0,402254,Rose-crested Blue Pipit,call,no score,13:30,2/8/2018,49,63
1,406171,Rose-crested Blue Pipit,call,A,7:48,6/7/2017,125,133
2,405901,Rose-crested Blue Pipit,call,A,12:00,2/8/2018,58,76
3,405548,Rose-crested Blue Pipit,song,A,11:00,3/10/2018,55,125
4,401782,Rose-crested Blue Pipit,song,A,6:00,6/29/2008,129,123


In [5]:
bird_names = pd.unique(data.English_name).tolist()
# len(bird_names)

In [6]:
dumping_site = (148,159)
data['Y']=data['Y'].str.replace('?', '')
data['Y']=pd.to_numeric(data['Y'], errors='ignore')

format1 = pd.to_datetime(data['Date'], errors = 'coerce', format = "%m/%d/%Y").dropna()
format2 = pd.to_datetime(data['Date'], errors = 'coerce', format = "%Y-%m-%d").dropna()
format3 = pd.to_datetime(data['Date'], errors = 'coerce', format = "%Y-%m-00").dropna()


data = data.drop(columns=['Date'])


data['Date'] = pd.concat([format1,format2,format3], axis = 0)
data =data[~data['Date'].isna()]

In [7]:
colors =[COLORS[bird_names.index(i)] for i in data.English_name.values.tolist()]
# colors_hex = ['#%02x%02x%02x'% c for c in colors]

In [8]:
data['color'] = colors

In [9]:
data_test = pd.read_csv("Test Birds Location.csv")

In [10]:
data_test.head()

,ID,X,Y
0,1,140,119
1,2,63,153
2,3,70,136
3,4,78,150
4,5,60,90


<div class="alert alert-block alert-success">
    <b>Bokeh Plotting</b>
</div>

In [11]:
def angry_birds_basic(data,data_test=data_test, ratio = 4):
    data_copy = data.copy()
    # Scaling the coordinates for the background image
    data_copy['X'] = data_copy['X'] * ratio
    data_copy['Y'] = data_copy['Y'] * ratio

    
    source_data = ColumnDataSource(data=dict(
        x=data_copy['X'],
        y=data_copy['Y'],
        x_disp=data['X'],
        y_disp=data['Y'],
        
    ))
    
    source_test_data = ColumnDataSource(data=dict(
        x = data_test[' X'] * ratio,
        y = data_test[' Y'] * ratio,
        x_disp=data_test[' X'],
        y_disp=data_test[' Y'],
    ))
    
    dumping_site = (148 * ratio, 159 * ratio)
    # Plot size
    plot_width = 200 * ratio
    # Adding Tools to the figure
    hover_tool = HoverTool(tooltips=[
        ('Position','(@x_disp, @y_disp)'),
    ])
    tools = [hover_tool, 'pan','wheel_zoom','reset','box_select','lasso_select',
            'zoom_in','zoom_out']
    # Creating the canvas(figure)
    fig = figure(x_range=(0,plot_width), y_range=(0,plot_width),plot_width = plot_width,
                 plot_height = plot_width, tools = tools)
    # Style of the plot - Removing the grid axis
    fig.xaxis.visible = False
    fig.xgrid.visible = False
    fig.yaxis.visible = False
    fig.ygrid.visible = False
    fig.tools.append(hover_tool)
    # Add the map to the canvas bg
    fig.image_url( url=["maps/map.jpg"],
             x=0, y=0, w=plot_width, h=plot_width, alpha = 0.8, anchor="bottom_left")
    # Plotting the dump site position
    fig.circle_x(dumping_site[0], dumping_site[1],size = 40, fill_color='red', fill_alpha = 0.4
                 , legend= "Dumping Site")
    # Plotting all the birds on the map
    fig.circle('x' ,'y',fill_color="blue", fill_alpha=0.7, source = source_data, size = 7)   
    fig.diamond('x','y', fill_color="orange", fill_alpha = 1 , source = source_test_data, size = 20) 
    # Showing the result
    show(fig)

In [12]:
angry_birds_basic(data)

<div class="alert alert-block alert-success">
    <b>Bokeh Plotting - Types</b>
</div>

In [13]:
data_test

,ID,X,Y
0,1,140,119
1,2,63,153
2,3,70,136
3,4,78,150
4,5,60,90
5,6,126,103
6,7,71,121
7,8,78,62
8,9,61,145
9,10,45,39


In [14]:
pred_gb = pd.read_csv("grad_boosting.csv")
pred_knn = pd.read_csv("pred_knn.csv")
pred_rf = pd.read_csv("pred_random_forest.csv")
data_test["gradBoost"] = pred_gb["English_name"]
data_test["RF"] = pred_rf["English_name"]
data_test["KNN"] = pred_knn["English_name"]

In [15]:
preds = ["Gradient Boosting: %s, Random Forest: %s, KNN: %s" % (
    a, b, c) for (a, b, c) in zip(data_test["gradBoost"], data_test["RF"], data_test["KNN"])]

In [16]:
data_test["name"] = preds

In [17]:
def angry_birds_type(data,data_test=data_test, ratio = 4):
    dumping_site = (148 * ratio, 159 *ratio)
    data_copy = data.copy()
    # Scaling the coordinates for the background image
    data_copy['X'] = data_copy['X'] * ratio
    data_copy['Y'] = data_copy['Y'] * ratio
    # Add the dumping site to the data for better visualization
    data_copy = data_copy.append({'English_name':'Dumping Site','X': dumping_site[0], 'Y':dumping_site[1],'color':'red'}
                     , ignore_index=True)
    # Bird names for the interactive legend
    bird_names = pd.unique(data_copy.English_name).tolist()
#     print(bird_names)
    # Plot size
    plot_width = 200 * ratio
    # Adding Tools to the figure
    hover_tool = HoverTool(tooltips=[
        ('Position','(@a, @b)'),
        ('Bird Name','@name'),
    ])
    tools = [hover_tool, 'pan','wheel_zoom','reset','lasso_select','box_zoom',
            'zoom_in','zoom_out']
    # Creating the canvas(figure)
    fig = figure(x_range=(0,plot_width), y_range=(0,plot_width),plot_width=plot_width,
                 plot_height = plot_width, tools = tools, toolbar_location="above")
    # Style of the plot - Removing the grid axis
    fig.xaxis.visible = False
    fig.xgrid.visible = False
    fig.yaxis.visible = False
    fig.ygrid.visible = False
    # Adding the hover tool
    fig.tools.append(hover_tool)
    # Add the map to the canvas bg
    fig.image_url( url=["maps/map.jpg"],
             x=0, y=0, w=plot_width, h=plot_width, alpha = 0.8, anchor="bottom_left")
    # Plotting the dump site position
    source_test_data = ColumnDataSource(data=dict(
        x = data_test[' X'] * ratio,
        y = data_test[' Y'] * ratio,
        a=data_test[' X'],
        b=data_test[' Y'],
        name = ["Test Data"] * data_test.shape[0],
    ))
    # Plotting all the birds on the map bu type for the interactive legend
    for n in bird_names:
        data_per_type = data_copy.loc[data_copy['English_name'] == n]
        source_per_type = ColumnDataSource(data=dict(
            x=data_per_type['X'],
            y=data_per_type['Y'],
            a=data_per_type['X']/ratio,
            b=data_per_type['Y']/ratio,
            name = data_per_type['English_name'],
            color = data_per_type['color'],
        ))
        
        if n != 'Dumping Site':
            fig.scatter('x' ,'y',fill_color='color',line_color='color', legend = 'name',
                   fill_alpha=0.8, size=5, source = source_per_type)
        else:
            fig.circle_x('x', 'y',size = 30, fill_color='color', fill_alpha = 0.4
                 , legend= 'name', source=source_per_type)
            
    # Drop the dumping site from the data_copy
    data_copy = data_copy.drop(data_copy.loc[data_copy['English_name'] == 'Dumping Site'].index)
    fig.asterisk('x','y', fill_color="blue",line_color="blue", fill_alpha = 1 , source = source_test_data, size = 30, legend = 'name') 
    # Adding interactive legend
    fig.legend.location = (0,0)
    fig.legend.click_policy="hide"
    fig.legend.label_text_font_size = "7pt"
    # Showing the result
    show(fig)

In [18]:
angry_birds_type(data)

<div class="alert alert-block alert-info">
    <b>Bokeh Plotting - Animations & Widgets</b>
</div>

In [19]:
data['Year'] = data['Date'].dt.year.astype(int)
data['Month'] = data['Date'].dt.month.astype(int)

In [20]:
data.head()

,File ID,English_name,Vocalization_type,Quality,Time,X,Y,Date,color,Year,Month
0,402254,Rose-crested Blue Pipit,call,no score,13:30,49,63,2018-02-08,#7DFB6D,2018,2
1,406171,Rose-crested Blue Pipit,call,A,7:48,125,133,2017-06-07,#7DFB6D,2017,6
2,405901,Rose-crested Blue Pipit,call,A,12:00,58,76,2018-02-08,#7DFB6D,2018,2
3,405548,Rose-crested Blue Pipit,song,A,11:00,55,125,2018-03-10,#7DFB6D,2018,3
4,401782,Rose-crested Blue Pipit,song,A,6:00,129,123,2008-06-29,#7DFB6D,2008,6


In [21]:
# Imports for the widgets
from bokeh.layouts import widgetbox, row, column
from bokeh.models.widgets import Dropdown, MultiSelect, DateSlider, Button
from bokeh.events import ButtonClick
from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler

In [24]:
def angry_birds_animated(data=data,data_test=data_test, ratio = 4):
    # Some preprocessing -----------------------------------------------------
    dumping_site = (148 * ratio, 159 *ratio)
    data_copy = data.copy()
    b_names = pd.unique(data_copy.English_name).tolist()
    # Scaling the coordinates for the background image
    data_copy['X'] = data_copy['X'] * ratio
    data_copy['Y'] = data_copy['Y'] * ratio
    # Adding some params for the plotting
    data_copy['marker'] = ['circle'] * data_copy.shape[0]
    data_copy['line'] = data_copy['color']
    data_copy['size'] = [10] * data_copy.shape[0]
    data_copy['alpha'] = [8] * data_copy.shape[0]
    # Add the dumping site to the data for better visualization
    data_copy = data_copy.append({'English_name':'Dumping Site','X': dumping_site[0], 'Y':dumping_site[1]
                                  ,'color':'red', 'marker':'circle_x'
                                  , 'size':30, 'line':'blue' ,'alpha':2, 'Year':-1,'Month':-1}
                     , ignore_index=True)
    # Source data
    source_data = ColumnDataSource(data=dict(
                x=data_copy['X'],
                y=data_copy['Y'],
                a=data_copy['X']/ratio,
                b=data_copy['Y']/ratio,
                year = data_copy['Year'],
                month = data_copy['Month'],
                name = data_copy['English_name'],
                color = data_copy['color'],
                marker = data_copy['marker'],
                size = data_copy['size'],
                line = data_copy['line'],
                alpha = data_copy['alpha']/10,
            ))
    # Add a Multi Select
    multi_select =  MultiSelect(title="Bird Type(s)", value=["All Birds"], options=['All Birds'] + b_names,
                               size=20)
    # Source for the plotting
    source = ColumnDataSource(data=dict(
                x=data_copy['X'],
                y=data_copy['Y'],
                a=data_copy['X']/ratio,
                b=data_copy['Y']/ratio,
                year = data_copy['Year'],
                month = data_copy['Month'],
                name = data_copy['English_name'],
                color = data_copy['color'],
                marker = data_copy['marker'],
                size = data_copy['size'],
                line = data_copy['line'],
                alpha = data_copy['alpha']/10,
                selected = data_copy['English_name'],
            ))
    # test birds
    # Plotting the dump site position
    source_test_data = ColumnDataSource(data=dict(
        x = data_test[' X'] * ratio,
        y = data_test[' Y'] * ratio,
        a=data_test[' X'],
        b=data_test[' Y'],
        name = data_test["name"],
        year = ["yyyy"] * data_test.shape[0],
        month = ["mm"] * data_test.shape[0],
    ))
    # Define callback functions to update & visulalize the data
    CallbackMultiSelect = CustomJS(args=dict(source_oj=source_data, source_plt = source), code="""
        var data = source_oj.data;
        var data_plt = source_plt.data;
        var f = cb_obj.value;
        if(Array.isArray(f)){
            // Callback for the multiselect
            f.push('Dumping Site');
            if(!f.includes('All Birds')){
                var types = data['name']; 
                var x= [],y= [],a= [],b= [],name= [],color= [],line= []
                ,size= [],alpha= [],marker= [], months=[], years=[],selected=[];
                for(i in types){
                    if(f.includes(types[i])){
                        x.push(data['x'][i]);
                        y.push(data['y'][i]);
                        a.push(data['a'][i]);
                        b.push(data['b'][i]);
                        name.push(data['name'][i]);
                        color.push(data['color'][i]);
                        marker.push(data['marker'][i]);
                        line.push(data['line'][i]);
                        alpha.push(data['alpha'][i]);
                        size.push(data['size'][i]);
                        months.push(data['month'][i]);
                        years.push(data['year'][i]);
                        selected.push(data['name'][i]);
                    }
                }
                var tmp = {
                        'x':x,
                        'y':y,
                        'a':a,
                        'b':b,
                        'name':name,
                        'marker':marker,
                        'line':line,
                        'alpha':alpha,
                        'size':size,
                        'color':color,
                        'month':months,
                        'year':years,
                        'selected':selected
                        }
                source_plt.data = tmp ;
                source_plt.change.emit();

            }
            else{
            source_plt.data = data ;
            source_plt.change.emit();
        }
    }
    """)
    
    # Generating the plot -----------------------------------------------------
    # Plot size
    plot_width = 200 * ratio
    # Adding Tools to the figure
    hover_tool = HoverTool(tooltips=[
        ('Position','(@a, @b)'),
        ('Bird Name','@name'),
        ('Data Taken in :','@month/@year'),
    ])
    tools = [hover_tool, 'pan','wheel_zoom','reset','lasso_select','box_zoom',
            'zoom_in','zoom_out']
    # Creating the canvas(figure)
    fig = figure(x_range=(0,plot_width), y_range=(0,plot_width),plot_width=plot_width,
                 plot_height = plot_width, tools = tools, toolbar_location="above",
                output_backend="webgl")
    
    # Style of the plot - Removing the grid axis
    fig.xaxis.visible = False
    fig.xgrid.visible = False
    fig.yaxis.visible = False
    fig.ygrid.visible = False
    
    # Add the map to the canvas bg
    fig.image_url( url=["maps/map.jpg"],
             x=0, y=0, w=plot_width, h=plot_width, alpha = 0.8, anchor="bottom_left")
     
    # Setting up the widgets -----------------------------------------------------
    multi_select.js_on_change('value',CallbackMultiSelect)
    # Add a slider for the date
    date_slider = DateSlider(title="Date of observation",value =data_copy['Date'].min(),
                             start = data_copy['Date'].min(),end = data_copy['Date'].max(),
                             step = 1, format="%b %Y")
    
    # Add Stop Button
        
    # Add Play Button
    stacked_layout = column(multi_select, date_slider)
    
    # Callback for the slider
    CallbackSlider = CustomJS(args=dict(source_oj = source_data, source_plt= source,end = date_slider.end,
                                       animate=True ), code="""
    // Fixed Values - Global
    const endDate = new Date(end);
    const endYear = endDate.getFullYear();
    const endMonth = endDate.getMonth()+1;

    const selectedTypes = new Set(source_plt.data['selected']);
    selectedTypes.add('Dumping Site');
    // -------------------------------------------------------
    // get the date
    var date = new Date(cb_obj.value);

    function update(date){
        var x= [],y= [],a= [],b= [],name= [],color= [],line= []
            ,size= [],alpha= [],marker= [], months=[],years=[];
        // get the month and the year

        month = date.getMonth()+1;
        year = date.getUTCFullYear();
        //---------------------
        var data = source_oj.data;
        yearUTC = data['year'];
        // --------------------
        for(i in yearUTC){
                if(yearUTC[i] == year || yearUTC[i] == -1){
                    if(month == data['month'][i] || data['month'][i] == -1){
                        if(selectedTypes.has(data['name'][i])){
                            x.push(data['x'][i]);
                            y.push(data['y'][i]);
                            a.push(data['a'][i]);
                            b.push(data['b'][i]);
                            name.push(data['name'][i]);
                            color.push(data['color'][i]);
                            marker.push(data['marker'][i]);
                            line.push(data['line'][i]);
                            alpha.push(data['alpha'][i]);
                            size.push(data['size'][i]);
                            months.push(data['month'][i]);
                            years.push(data['year'][i]);
                        }
                    }
                }
            }
        var tmp = {
            'x':x,
            'y':y,
            'a':a,
            'b':b,
            'name':name,
            'marker':marker,
            'line':line,
            'alpha':alpha,
            'size':size,
            'color':color,
            'month':months,
            'year':years,
            'selected':selectedTypes
            }
        //console.log(data);
        source_plt.data = tmp ;
        source_plt.change.emit();
    }
    //var t0 = performance.now();
    update(date);
    var step = 1000;
    //var t1 = performance.now();
    //console.log(t1-t0);
    date = new Date(date.setMonth(date.getMonth()+1));
    if(animate){
        //console.log(date);
        // ******************************
        /*
        window.setInterval(async function(){
            if(date <= endDate){
                cb_obj.value = date;
                update(date);
                date = new Date(date.setMonth(date.getMonth()+1));
                i++;
            }
            else{
                clearInterval(refresh);
            }
           
            
        },step);*/
        
        
    }
    
    """)
    
    
    date_slider.js_on_change('value',CallbackSlider)
    fig.scatter('x' ,'y',fill_color='color',line_color='line',fill_alpha='alpha', size='size', source = source, marker='marker')
    layout = row(stacked_layout, fig, sizing_mode="scale_height")
    fig.asterisk('x','y', fill_color="blue",line_color="blue", fill_alpha = 1 , source = source_test_data, size = 30) 
    # Show the whole layout-----------------------------------------------------
    show(layout)


In [25]:
angry_birds_animated()